## Họ và tên : Trần Anh Thái
## MSSV : 22H1320007

In [1]:
#Tạo dataframe 
import pandas as pd

with open('SMSSpamCollection', 'r', encoding='UTF-8') as file:
    email_data = file.read()
    
lines = email_data.strip().split("\n")
data = [line.split("\t", 1) for line in lines]
df = pd.DataFrame(data, columns=["label", "content"])
df


,label,content
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('puntk_tab')

def preprocess_text(text):
    text = text.lower()
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords.words('english')] 
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading puntk_tab: Package 'puntk_tab' not found in
[nltk_data]     index


In [3]:
df['message'] = df['content'].apply(preprocess_text)

In [4]:
df['message']

0       go jurong point, crazy.. available bugis n gre...
1                           ok lar... joking wif u oni...
2       free entry 2 wkly comp win fa cup final tkts 2...
3               u dun say early hor... u c already say...
4                 nah think goes usf, lives around though
                              ...                        
5569    2nd time tried 2 contact u. u £750 pound prize...
5570                         ü b going esplanade fr home?
5571             pity, * mood that. so...any suggestions?
5572    guy bitching acted like i'd interested buying ...
5573                                      rofl. true name
Name: message, Length: 5574, dtype: object

In [5]:
#Phân chia tập train, test theo tỉ lệ 7:3
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], train_size=0.7, test_size=0.3, random_state=0)

print(X_train.shape, X_test.shape)

(3901,) (1673,)


In [6]:
#Vector hóa dữ liệu text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#Khởi tạo CountVectorizer
count_vector = CountVectorizer()

X_train_Count = count_vector.fit_transform(X_train).toarray()
X_test_Count = count_vector.transform(X_test).toarray()

#Khởi tạo TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()



In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

names = ['Support Vector Machines (SVM)',
         'Logistic Regression',
         'Naive Bayes']

classifier = [
    SVC(kernel = 'rbf'),
    LogisticRegression(),
    MultinomialNB()
]

In [10]:
print('Vector hóa dữ liệu text dùng CountVectorizer')
for name, clf in zip(names, classifier):
    print(f'Score of {name} : {clf.fit(X_train_Count, y_train).score(X_test_Count, y_test)}')

print('\nVector hóa dữ liệu text dùng TfidfVectorizer')
for name, clf in zip(names, classifier):
    print(f'Score of {name} : {clf.fit(X_train_tfidf, y_train).score(X_test_tfidf, y_test)}')

Vector hóa dữ liệu text dùng CountVectorizer
Score of Support Vector Machines (SVM) : 0.9760908547519426
Score of Logistic Regression : 0.9760908547519426
Score of Naive Bayes : 0.9838613269575612

Vector hóa dữ liệu text dùng TfidfVectorizer
Score of Support Vector Machines (SVM) : 0.9772863120143455
Score of Logistic Regression : 0.9557680812910938
Score of Naive Bayes : 0.9641362821279139


In [22]:
def test_model_use_CountVectorizer(message, names, classifier):
    message_vector = count_vector.transform([message]).toarray()
    for name, clf in zip(names, classifier):
        print(f'Prediction of {name} : {clf.predict(message_vector)}')
        
def test_model_use_TfidfVectorizer(message, names, classifier):
    message_vector = tfidf_vectorizer.transform([message]).toarray()
    for name, clf in zip(names, classifier):
        print(f'Prediction of {name} : {clf.predict(message_vector)}')

In [23]:
email_input = input("Nhập nội dung email để dự đoán: \n")
print('Nội dung email : ')
print(email_input)

print(f'Vector hóa dữ liệu text dùng CountVectorizer')
try:
    test_model_use_CountVectorizer(email_input, names, classifier)

    print('\nVector hóa dữ liệu text dùng TfidfVectorizer')
    test_model_use_TfidfVectorizer(email_input, names, classifier)
except Exception as e:
    print(e)

Nội dung email : 
No. I meant the calculation is the same. That  &lt;#&gt; units at  &lt;#&gt; . This school is really expensive. Have you started practicing your accent. Because its important. And have you decided if you are doing 4years of dental school or if you'll just do the nmde exam.
Vector hóa dữ liệu text dùng CountVectorizer
Prediction of Support Vector Machines (SVM) : ['ham']
Prediction of Logistic Regression : ['ham']
Prediction of Naive Bayes : ['ham']

Vector hóa dữ liệu text dùng TfidfVectorizer
Prediction of Support Vector Machines (SVM) : ['ham']
Prediction of Logistic Regression : ['ham']
Prediction of Naive Bayes : ['ham']
